In [1]:
import asyncio
import os
import time
import random
from typing import Dict, List, Optional, Tuple

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.cache_utils import DynamicCache, StaticCache

from tqdm import tqdm

from shared import (
    MessageChannel,
    PrefillRequest,
    PrefillResponse,
    PrefillBatchRequest,
    PrefillBatchResponse,
    ResetRequest,
    VerifyRequest,
    VerifyResponse,
    VerifyBatchRequest,
    VerifyBatchResponse,
    VerifyResponseItem,
)

import torch.nn as nn
import numpy as np

from dotenv import load_dotenv
load_dotenv()

/Users/frank/gpu_mode/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
BASE_MODEL = os.environ.get("HF_BASE_MODEL", "meta-llama/Llama-3.1-8B")
TOP_K = int(os.environ.get("HF_TOP_K", "20"))
ATTN_IMPL_ENV = os.environ.get("HF_ATTN_IMPL", "").strip()  # e.g., "flash_attention_2" if available

In [3]:
DEVICE = torch.device('mps')
DTYPE = torch.float16

In [4]:
hf_token = os.environ.get("HF_TOKEN", None)
from_kwargs = {
    "dtype": DTYPE,
    "device_map": None,           # keep single process; move to one device below
    "low_cpu_mem_usage": True,
    "token": hf_token,
    "local_files_only": True,
}
if ATTN_IMPL_ENV:
    from_kwargs["attn_implementation"] = ATTN_IMPL_ENV

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    **from_kwargs,
).to(DEVICE) # type: ignore

tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL, 
    use_fast=True, 
    token=hf_token, 
    local_files_only=True,
)

PAD_ID = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else (
    tokenizer.eos_token_id if tokenizer.eos_token_id is not None else 0
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:14<00:00,  3.51s/it]


## Prefill Stage

In [11]:
from typing import Any

prompts_str = [
    # "def bubble_sort(x: list[int]):",
    "What is the meaning of life? Well, ",
]

tokens: list[list[int]] = [tokenizer.encode(prompt) for prompt in prompts_str]

In [12]:
@torch.inference_mode()
def prefill(model: nn.Module, tokens: list[list[int]]):
    max_len = max(len(x) for x in tokens)
    padded = [[PAD_ID] * (max_len - len(prompt)) + prompt for prompt in tokens]  # is pad id 0 correct?
    x = torch.tensor(padded, dtype=torch.long, device=DEVICE)

    cache = DynamicCache(
        config=model.config, 
    )

    outputs = model(
        input_ids=x, 
        past_key_values=cache, 
        use_cache=True,
    )

    return outputs.past_key_values

cache = prefill(model, tokens)

print(tokens)

[[128000, 3923, 374, 279, 7438, 315, 2324, 30, 8489, 11, 220]]


In [13]:
def rollback_dynamic_per_row_simple(cache: DynamicCache, tokens: list[list[int]], r: list[int]):
    """
    Roll back r[i] tokens for each batch row i in a DynamicCache.
    The output cache maintains the same sequence length as the input, padding with zeros where needed.
    """
    assert cache.layers[0].keys is not None and cache.layers[0].values is not None
    assert all([x >= 0 for x in r])
    B = cache.layers[0].keys.shape[0]
    device = cache.layers[0].keys.device
    dtype = cache.layers[0].keys.dtype

    # Prepare destination cache
    dst = DynamicCache()

    for layer in range(len(cache)):
        K = cache.layers[layer].keys
        V = cache.layers[layer].values
        assert K is not None and V is not None

        _, H, S, D = K.shape

        K_new = K.new_zeros((B, H, S, D))
        V_new = V.new_zeros((B, H, S, D))

        # Copy per row
        for i in range(B):
            keep = S - r[i]
            if keep <= 0:
                continue
            # surviving tokens are the first 'keep' positions (earliest..latest-rollback)
            K_src = K[i, :, :keep, :]
            V_src = V[i, :, :keep, :]

            # right-aligned → write to the right, pad on the left implicitly
            start = S - keep
            K_new[i, :, start:, :] = K_src
            V_new[i, :, start:, :] = V_src
            # print(K_new[i, 0, :, 0])

        # print(dst.layers[layer].keys[i, 0, :, 0])
        dst.update(K_new, V_new, layer)

    tokens = [x[:len(x) - trim] for x, trim in zip(tokens, r)]

    return dst, tokens

print(cache.layers[0].keys[0, 0, :, 0])
cache, tokens = rollback_dynamic_per_row_simple(cache, tokens, [4])
print(cache.layers[0].keys[0, 0, :, 0])

tensor([ 0.4468,  1.6230, -3.5273, -4.6562, -2.8164,  3.0645,  8.5391,  2.1406,
        -3.7188, -2.7695, -2.2598], device='mps:0', dtype=torch.float16)
tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.4468,  1.6230, -3.5273, -4.6562,
        -2.8164,  3.0645,  8.5391], device='mps:0', dtype=torch.float16)


In [ ]:
@torch.inference_mode()
def generate_step(
    model: nn.Module,
    cache: DynamicCache,
    tokens: list[list[int]],
    lengths: torch.LongTensor,
):
    x = torch.tensor(tokens, dtype=torch.long, device=DEVICE).view(-1, 1)

    B = lengths.size(0)
    S_prev = cache.layers[0].keys.shape[2]

    # Use int64 (not bool) on MPS; build past + current token mask
    attn_mask = torch.zeros((B, S_prev + 1), dtype=torch.long, device=DEVICE)

    starts = (S_prev - lengths).clamp_min(0)                # (B,)
    idx = torch.arange(S_prev, device=DEVICE)[None, :]      # (1, S_prev)
    attn_mask[:, :-1] = (idx >= starts[:, None]).to(torch.long)
    attn_mask[:, -1] = 1

    pos_ids = lengths.view(B, 1)  # new token's RoPE position (0..L_i)

    out = model(
        input_ids=x,
        past_key_values=cache,
        use_cache=True,
        attention_mask=attn_mask,  # int mask
        position_ids=pos_ids,
    )

    next_tok = out.logits[:, -1].argmax(dim=-1).tolist()
    lengths = lengths + 1
    return [[t] for t in next_tok], lengths




suffix_text = [
    '?',
]
generated = [tokenizer.encode(x)[1:] for x in suffix_text]
print(generated)

lengths = torch.tensor([len(x) for x in tokens], dtype=torch.long, device=model.device)
print(lengths)

for _ in tqdm(range(20)):
    tokens: list[list[int]] = [x + y for x, y in zip(tokens, generated)]
    print(tokens)

    generated, lengths = generate_step(model, cache, generated, lengths)
    print([tokenizer.decode(x) for x in generated])

print(tokenizer.decode(tokens[0]))

[[30]]
tensor([7], device='mps:0')


  0%|          | 0/20 [00:00<?, ?it/s]

[[128000, 3923, 374, 279, 7438, 315, 2324, 30]]


  5%|▌         | 1/20 [00:01<00:31,  1.68s/it]

[' This']
[[128000, 3923, 374, 279, 7438, 315, 2324, 30, 1115]]


...

 80%|████████  | 16/20 [00:03<00:00, 10.15it/s]

[' for']
[[128000, 3923, 374, 279, 7438, 315, 2324, 30, 1115, 374, 264, 3488, 430, 706, 87420, 61787, 11, 90602, 5493, 11, 323, 14248, 369]]
[' centuries']
[[128000, 3923, 374, 279, 7438, 315, 2324, 30, 1115, 374, 264, 3488, 430, 706, 87420, 61787, 11, 90602, 5493, 11, 323, 14248, 369, 24552]]
['.']
[[128000, 3923, 374, 279, 7438, 315, 2324, 30, 1115, 374, 264, 3488, 430, 706, 87420, 61787, 11, 90602, 5493, 11, 323, 14248, 369, 24552, 13]]


100%|██████████| 20/20 [00:03<00:00,  5.77it/s]

[' There']
[[128000, 3923, 374, 279, 7438, 315, 2324, 30, 1115, 374, 264, 3488, 430, 706, 87420, 61787, 11, 90602, 5493, 11, 323, 14248, 369, 24552, 13, 2684]]
[' are']
[[128000, 3923, 374, 279, 7438, 315, 2324, 30, 1115, 374, 264, 3488, 430, 706, 87420, 61787, 11, 90602, 5493, 11, 323, 14248, 369, 24552, 13, 2684, 527]]
[' many']
<|begin_of_text|>What is the meaning of life? This is a question that has puzzled philosophers, theologians, and scientists for centuries. There are


IndexError: list index out of range